Funktionen aus US1

### Änderung an `find_tsv_files`

In der ursprünglichen Version wurden **alle `.tsv`-Dateien** rekursiv gefunden – einschließlich bereits bereinigter Dateien mit dem Suffix `_cleaned.tsv`.

In der neuen Version werden solche bereits bereinigten Dateien **ausgeschlossen**, um zu verhindern, dass sie erneut verarbeitet und dadurch doppelt gespeichert werden (z. B. `_cleaned_cleaned.tsv`).

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import display

# Rekursive Suche nach .tsv-Dateien, wobei bereits bereinigte Dateien ausgeschlossen werden
def find_tsv_files(directory, exclude_cleaned=True):
    tsv_files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tsv'):
                if exclude_cleaned and file.endswith('_cleaned.tsv'):
                    continue  # Skip already-cleaned files
                full_path = os.path.join(root, file)
                tsv_files_list.append(full_path)
    return tsv_files_list

Um die Ergebnisse von User Story 2 darzustellen, wurde auf die Visualisierung aus User Story 1 verzichtet.

In [2]:
# Funktion zum markieren von Ausreißern
def mark_outliers(file_path, method='iqr', threshold=1.5):

    try:
        # Einlesen der TSV-Datei
        df = pd.read_csv(file_path, sep='\t')

        # Einlesen der Spaltennamen
        gene_col = df.columns[0] # erste Spalte ist immer das Gen
        symbol_col = df.columns[-1] # letzte Spalte ist immer das Symbol
        entity_col = df.columns[-2] # vorletzte Spalte ist immer die Entity

        # Count-Spalten extrahieren
        count_cols = df.columns[1:-2]

        # DataFrame für Ergebnisse initialisieren
        results_df = df.copy()
        results_df['is_outlier'] = False
        results_df['outlier_info']  = ""

        # Gruppierung nach Entity (host/phage)
        host_mask = df[entity_col] == 'host'
        phage_mask = df[entity_col] == 'phage'

        # Anzahl der Host- und Phagen-Gene
        #print(f"Identifizierte Gene: {sum(host_mask)} Host-Gene, {sum(phage_mask)} Phagen-Gene")

       # Ausreißer für Host-Gene identifizieren
        if sum(host_mask) > 0:
            host_outliers = detect_outliers(df[host_mask], count_cols, method, threshold)
            for idx in host_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Host-Outlier; "

        # Ausreißer für Phagen-Gene identifizieren
        if sum(phage_mask) > 0:
            phage_outliers = detect_outliers(df[phage_mask], count_cols, method, threshold)
            for idx in phage_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Phagen-Outlier; "

        return results_df

    except Exception as e:
        print(f"Fehler {e}")
        return None

# Hilfsfunktion zur Ausreißererkennung
def detect_outliers(df, count_cols, method='iqr', threshold=1.5):
    outlier_indices = set()

    if method == 'iqr':
        # Interquartilsabstand-Methode
        for col in count_cols:
            Q05 = df[col].quantile(0.05)
            Q1 = df[col].quantile(0.25)
            #print(f"Q1: {Q1}")
            Q3 = df[col].quantile(0.75)
            #print(f"Q3: {Q3}")
            IQR = Q3 - Q1
            #print(f"IQR: {IQR}")
            lower_bound = Q1 - threshold * IQR
            #print(f"Lower Bound: {lower_bound}")
            upper_bound = Q3 + threshold * IQR
            #print(f"Upper Bound: {upper_bound}")
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.update(outliers)

         # Berechne das 0.05-Quantil für jede Spalte
        quantiles_05 = {col: df[col].quantile(0.05) for col in count_cols}

        # Prüfe für jede Zeile, ob alle Werte kleiner als das 0.05-Quantil sind
        for idx, row in df.iterrows():
            if all(row[col] < quantiles_05[col] for col in count_cols):
                outlier_indices.add(idx)

    elif method == 'zscore':
        # Z-Score-Methode
        for col in count_cols:
            z_scores = stats.zscore(df[col], nan_policy='omit')
            outliers = df[abs(z_scores) > threshold].index
            outlier_indices.update(outliers)

    return outlier_indices


### Datenbereinigung  

Als Nächstes definieren wir eine Funktion `clean_outlier_samples`, die potenzielle Ausreißer erkennt und entfernt. Diese Funktion basiert auf der vorherigen Methode zur Ausreißererkennung (`mark_outliers`), entfernt im Anschluss die betroffenen Zeilen sowie die dazugehörigen Spalten `is_outlier` und `outlier_info`.

Die bereinigte Version jeder Datei wird unter demselben Namen mit dem Suffix `_cleaned.tsv` **im ursprünglichen Verzeichnis** der Datei gespeichert. Dadurch bleibt der ursprüngliche Datensatz unverändert erhalten, und die finalen, bereinigten Daten befinden sich direkt am ursprünglichen Speicherort.

Für die gleichzeitige Bereinigung mehrerer Dateien definieren wir zusätzlich eine Batch-Funktion `batch_clean_all_tsv`. Diese durchläuft rekursiv alle `.tsv`-Dateien in einem angegebenen Verzeichnis (einschließlich Unterverzeichnissen), bereinigt sie nacheinander mit der beschriebenen Methode und speichert die bereinigten Versionen jeweils im selben Verzeichnis wie die Originaldateien.

Um die bereinigten Dateien zu überprüfen, verwenden wir die Funktion `preview_tsv_files`. Diese sucht rekursiv nach allen Dateien mit dem Suffix `_cleaned.tsv` und zeigt eine Vorschau der ersten Zeilen an.

In [3]:
import glob

#Funktion zur Datenbereinigung
def clean_outlier_samples(file_path, method='iqr', threshold=1.5):

    cleaned_df = mark_outliers(file_path, method=method, threshold=threshold)

    if cleaned_df is None:
        print(f"Fehler beim Verarbeiten der Datei: {file_path}")
        return

    # Count outliers
    num_outliers = cleaned_df["is_outlier"].sum()

    # Remove outliers and drop the helper columns
    cleaned_df = cleaned_df[~cleaned_df["is_outlier"]].copy()
    cleaned_df.drop(columns=["is_outlier", "outlier_info"], inplace=True, errors='ignore')

    # Save to original directory
    original_dir = os.path.dirname(file_path)
    file_name = os.path.basename(file_path).replace(".tsv", "_cleaned.tsv")
    save_path = os.path.join(original_dir, file_name)
    cleaned_df.to_csv(save_path, sep='\t', index=False)

    print(f"{num_outliers} Ausreißer wurden entfernt → {file_name}")
    print(f"Finalisierte Datei gespeichert: {save_path}")

    return cleaned_df


def batch_clean_all_tsv(input_dir, method='iqr', threshold=1.5):
    tsv_files = find_tsv_files(input_dir)

    if not tsv_files:
        print(f"Keine TSV-Dateien gefunden in: {input_dir}")
        return

    print(f"{len(tsv_files)} Dateien werden verarbeitet...\n")
    for file_path in tsv_files:
        clean_outlier_samples(
            file_path=file_path,
            method=method,
            threshold=threshold
        )
    print("\nAlle Dateien wurden verarbeitet.")

Zum Abschluss der Datenbereinigung möchten wir einen schnellen Überblick über die bereinigten Datensätze erhalten. Dazu verwenden wir die Funktion `preview_tsv_files`, die eine definierte Anzahl an Zeilen aus allen `.tsv`-Dateien in einem angegebenen Verzeichnis anzeigt.  

Diese Vorschau hilft dabei, die Struktur der Dateien zu kontrollieren und sicherzustellen, dass die Spalten `is_outlier` und `outlier_info` (sofern entfernt) nicht mehr enthalten sind.  

Zunächst rufen wir `batch_clean_all_tsv()` auf, um alle `.tsv`-Dateien im Eingabeverzeichnis zu bereinigen. Anschließend verwenden wir `preview_tsv_files`, um die bereinigten Dateien (`_cleaned.tsv`) zu betrachten.  

In [4]:
def preview_tsv_files(directory, suffix="_cleaned.tsv", preview_rows=3):
    found_files = []

    # Recursively search all subdirectories for files with the given suffix
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(suffix):
                found_files.append(os.path.join(root, file))

    if not found_files:
        print(f"Keine Dateien mit dem Suffix '{suffix}' gefunden in: {directory}")
        return

    # Display each cleaned file found
    for path in found_files:
        print(f"{path}")
        df = pd.read_csv(path, sep='\t')
        display(df.head(preview_rows))


In [5]:
input_dir = '../data'

batch_clean_all_tsv(input_dir)

print()
print("Vorschau der bereinigten Dateien (ohne 'is_outlier' und 'outlier_info')")
preview_tsv_files(input_dir, "_cleaned.tsv")

22 Dateien werden verarbeitet...

805 Ausreißer wurden entfernt → Leskinen_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/leskinen_2016/Leskinen_full_raw_counts_cleaned.tsv
747 Ausreißer wurden entfernt → Li_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/li_2020/Li_full_raw_counts_cleaned.tsv
644 Ausreißer wurden entfernt → Finstrlova_SH1000_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/finstrlova_2022/Finstrlova_SH1000_full_raw_counts_cleaned.tsv
662 Ausreißer wurden entfernt → Finstrlova_Newman_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/finstrlova_2022/Finstrlova_Newman_full_raw_counts_cleaned.tsv
576 Ausreißer wurden entfernt → Yang_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/yang_2019/Yang_full_raw_counts_cleaned.tsv
438 Ausreißer wurden entfernt → Kuptsov_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/kuptsov_2022/Kuptsov_full_raw_counts_cleaned.tsv

,Geneid,0,2,5,10,15,21,28,35,42,49,Entity,Symbol
0,gene-Y11_RS02515,869,354,441,593,579,482,427,475,501,352,host,xyeB
1,gene-Y11_RS16005,6654,4404,4482,5204,4829,3641,3149,3479,2541,2569,host,hemY
2,gene-Y11_RS11320,601,345,314,357,482,228,266,359,243,342,host,hypB


../data/li_2020/Li_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,0_R3,30_R1,30_R2,30_R3,45_R1,45_R2,45_R3,75_R1,75_R2,75_R3,135_R1,135_R2,135_R3,Entity,Symbol
0,gene-FTB24_03575,117,120,181,54,118,103,105,101,93,79,118,127,149,220,261,host,dpsA
1,gene-FTB24_07525,17,14,127,16,8,57,20,20,33,18,26,48,32,139,173,host,asrB
2,gene-FTB24_13070,24,13,111,17,21,46,21,26,25,22,26,40,41,145,104,host,spoIIIAE


../data/finstrlova_2022/Finstrlova_Newman_full_raw_counts_cleaned.tsv


...

,Geneid,bim-phage-35_R1,bim-phage-35_R2,bim-phage-35_R3,bim-phage-35_R4,bim-phage-35_R5,bim-phage-60_R1,bim-phage-60_R2,bim-phage-60_R3,bim-phage-60_R4,bim-phage-60_R5,bim-phage-120_R1,bim-phage-120_R2,bim-phage-120_R3,bim-phage-120_R4,bim-phage-120_R5,Entity,Symbol
0,gene-R0792_RS03340,1,4,5,4,9,5,0,4,5,3,6,8,2,3,6,host,pyk
1,gene-R0792_RS18895,29,62,98,43,164,93,69,99,102,61,83,157,79,85,77,host,gene-R0792_RS18895
2,gene-R0792_RS12615,46,55,101,74,126,85,57,81,78,99,69,132,67,60,76,host,gene-R0792_RS12615


../data/meaden_2021/Meaden_WT_full_raw_counts_cleaned.tsv


,Geneid,wt-phage-35_R1,wt-phage-35_R3,wt-phage-35_R4,wt-phage-35_R5,wt-phage-60_R1,wt-phage-60_R2,wt-phage-60_R3,wt-phage-60_R4,wt-phage-120_R1,wt-phage-120_R2,wt-phage-120_R3,wt-phage-120_R4,Entity,Symbol
0,gene-R0792_RS21710,57,52,80,53,14,40,56,46,88,44,60,42,host,aroE
1,gene-R0792_RS24955,111,118,135,102,47,150,113,79,135,67,133,91,host,gene-R0792_RS24955
2,gene-R0792_RS22780,4,4,6,4,0,5,7,5,11,4,7,8,host,gene-R0792_RS22780


../data/zhong_2020/Zhong_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,0_R3,6_R1,6_R2,6_R3,12_R1,12_R2,12_R3,18_R1,18_R2,18_R3,Entity,Symbol
0,gene-PA3733a,231,281,367,151,345,178,64,112,99,117,125,184,host,gene-PA3733a
1,gene-PA2175,0,0,0,0,0,0,0,0,0,0,0,0,host,gene-PA2175
2,gene-PA3190,486,450,561,145,122,338,124,96,646,149,112,376,host,gene-PA3190
